In [1]:
import pandas as pd

In [2]:
# to print out all the outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [3]:
df = pd.read_csv('../data/amz_uk_processed_data.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2222742 entries, 0 to 2222741
Data columns (total 10 columns):
 #   Column             Dtype  
---  ------             -----  
 0   asin               object 
 1   title              object 
 2   imgUrl             object 
 3   productURL         object 
 4   stars              float64
 5   reviews            int64  
 6   price              float64
 7   isBestSeller       bool   
 8   boughtInLastMonth  int64  
 9   categoryName       object 
dtypes: bool(1), float64(2), int64(2), object(5)
memory usage: 154.7+ MB


In [5]:
df1 = df.loc[df["reviews"].notnull()]  # has reviews only

In [6]:
df2 = df1.loc[df1["isBestSeller"] == True]  # best sellers only

In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6018 entries, 1 to 2222598
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   asin               6018 non-null   object 
 1   title              6018 non-null   object 
 2   imgUrl             6018 non-null   object 
 3   productURL         6018 non-null   object 
 4   stars              6018 non-null   float64
 5   reviews            6018 non-null   int64  
 6   price              6018 non-null   float64
 7   isBestSeller       6018 non-null   bool   
 8   boughtInLastMonth  6018 non-null   int64  
 9   categoryName       6018 non-null   object 
dtypes: bool(1), float64(2), int64(2), object(5)
memory usage: 476.0+ KB


In [8]:
df2["categoryName"].value_counts()

categoryName
Health & Personal Care                              485
Grocery                                             313
Sports & Outdoors                                   303
Pet Supplies                                        284
Power & Hand Tools                                  243
Baby                                                188
Women                                               183
Arts & Crafts                                       163
Men                                                 147
Hardware                                            146
Storage & Organisation                              142
Large Appliances                                    139
PC & Video Games                                    136
Customers' Most Loved                               109
Office Supplies                                     107
Building Supplies                                    93
Agricultural Equipment & Supplies                    90
Boys                               

In [9]:
df3 = df2.loc[df2["categoryName"] == "Health & Personal Care"]  # filter to single category

In [10]:
dfx = df3[['title',
          'stars',
          'reviews',
          'price',
          'boughtInLastMonth'
        ]].reset_index()

In [11]:
dfx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485 entries, 0 to 484
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              485 non-null    int64  
 1   title              485 non-null    object 
 2   stars              485 non-null    float64
 3   reviews            485 non-null    int64  
 4   price              485 non-null    float64
 5   boughtInLastMonth  485 non-null    int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 22.9+ KB


In [12]:
dfx.head()

,index,title,stars,reviews,price,boughtInLastMonth
0,1252284,Nature Spell Rosemary Oil for Hair & Skin 150ml – Rosemary Oil for Hair Growth – Treat Dry Damaged Hair to Target Hair Loss – Body Oil for Dry Skin – Made in the UK,4.5,3700,7.98,10000
1,1252286,"Nizoral Anti-dandruff Shampoo, Treats and Prevents Dandruff, Suitable for Dry Flaky and Itchy Scalp, Contains Ketoconazole - 60ml",4.6,27110,6.82,20000
2,1252287,"Sudafed Blocked Nose Spray, Relief from Congestion Caused by Head Cold and Allergies, Sinusitis, Helps Clear The Nasal Passage, Lasts Up to 10 Hours and Gets to Work in 2 Minutes, 15 Ml",4.7,14689,4.00,20000
3,1252288,"Step Ahead Covid-19 Lateral Flow Testing Kit | Pack Of 5 |, White",4.4,2833,6.95,20000
4,1252289,"Omega 3 Fish Oil 2000mg, EPA 660mg DHA 440mg per Daily Serving. 120 Capsules (2 Months Supply). Supports Heart, Brain Function and Eye Health. 2 Capsules Per Serving",4.6,11922,9.97,10000


In [13]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# Make sure you have your OpenAI key set up as the OPENAI_API_KEY environment variable, or set it manually

# Set the OpenAI API key env variable manually
# os.environ["OPENAI_API_KEY"] = "<your_api_key>"

load_dotenv()
api_key = os.getenv("API_KEY")
client = OpenAI(api_key=api_key)

True

In [14]:
# ENTITY TYPES:
entity_types = {
    "product": None,
    "type": None,
    "brand": None,
    "characteristic": None,
    "color": None,
    "color_group": None,
    "age_group": None,
    "gender_group": None,
    "benefits": None,
    "size": None,
    "origin": None,
}

In [15]:
system_prompt = """
You are an agent specialized in finding entities in online product descriptions.  
The user will give you a product description.  
Your task is to identify entities from the product description.
Use only the following entities:
{entity_types}

You must return a JSON output containing a list of values for every type of entity found.
Use only the following keys: "product", "type", "brand", "characteristic", "color",
"color_group", "age_group", "gender_group", "benefits", "size", "origin".

If you cannot find an entity type, return an empty array for this entity.  
If you found one entity of this type, return an array with one value.  
If you found 2 entities of this type, return an array with 2 values.  
Et cetera.  

Only use lower cases letters when defining entities values, and remove adjectives and specificities from values to try and have the simplest words or groups of words.

--> Beginning of example

# Product Description
"Nature Spell Rosemary Oil for Hair & Skin 150ml – Rosemary Oil for Hair Growth – 
Treat Dry Damaged Hair to Target Hair Loss – Body Oil for Dry Skin – Made in the UK"

################

# Output
[
    {{
        "product": ["rosemary oil"],
        "type": ["body oil"],
        "brand": ["nature spell"],
        "characteristic": [],
        "color": [],
        "color_group": [],
        "age_group": [],
        "gender_group": [],
        "benefits": ["treats dry damaged hair", "treat hair loss"],
        "size": ["150ml"],
        "origin": ["uk"],
    }}
]

--> End of example
"""

In [16]:
def extract_entities(text, model="gpt-3.5-turbo"):
   completion = client.chat.completions.create(
        model=model,
        temperature=0,
       response_format= {
           "type": "json_object"
       },
        messages=[
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": text
        }
        ]
    )

   return completion.choices[0].message.content 

In [17]:
title = "Nizoral Anti-dandruff Shampoo, Treats and Prevents Dandruff, Suitable for Dry Flaky and Itchy Scalp, Contains Ketoconazole - 60ml"
print(extract_entities(title))

{
    "product": ["anti-dandruff shampoo"],
    "type": ["shampoo"],
    "brand": ["nizoral"],
    "characteristic": ["anti-dandruff", "suitable for dry flaky and itchy scalp"],
    "color": [],
    "color_group": [],
    "age_group": [],
    "gender_group": [],
    "benefits": ["treats dandruff", "prevents dandruff"],
    "size": ["60ml"],
    "origin": []
}


In [18]:
import json

data_entities = []
for i, row in dfx.head(100).iterrows():
    print(f"#{i} - {row['title'][:50]}")
    entities = json.loads(extract_entities(row['title']))
    product_data_string = row.to_json(orient='columns')
    product_data = json.loads(product_data_string)
    product_data.update(entities)
    data_entities.append(product_data)

#0 - Nature Spell Rosemary Oil for Hair & Skin 150ml – 
#1 - Nizoral Anti-dandruff Shampoo, Treats and Prevents
#2 - Sudafed Blocked Nose Spray, Relief from Congestion
#3 - Step Ahead Covid-19 Lateral Flow Testing Kit | Pac
#4 - Omega 3 Fish Oil 2000mg, EPA 660mg DHA 440mg per D
#5 - Vitabiotics Ultra Vitamin D Tablets 1000IU Optimum
#6 - Remington D3190 Ionic Conditioning Hair Dryer for 
#7 - Apple Cider Vinegar Gummies with The Mother 1000mg
#8 - Philips OneBlade Replacement Blade - Pack of 3 (1 
#9 - Clearblue Pregnancy Test Double-Check & Date Combo
#10 - Dulcolax Adult 5 mg Gastro-resistant Tablets - Ove
#11 - Finish All in 1 Max Dishwasher Tablets Lemon Scent
#12 - Canesten Thrush Combi Internal & External Creams f
#13 - Oral-B Pro Cross Action Electric Toothbrush Head, 
#14 - RENPHO Digital Bathroom Scales for Body Weight, We
#15 - Collagen Powder, Gold Standard Bovine Collagen Pep
#16 - Otex Express Ear Drops, Clinically Proven Ear Wax 
#17 - Oral-B Vitality Pro Electric Toothb

In [19]:
type(data_entities)

list

In [20]:
data_entities

[{'index': 1252284,
  'title': 'Nature Spell Rosemary Oil for Hair & Skin 150ml – Rosemary Oil for Hair Growth – Treat Dry Damaged Hair to Target Hair Loss – Body Oil for Dry Skin – Made in the UK',
  'stars': 4.5,
  'reviews': 3700,
  'price': 7.98,
  'boughtInLastMonth': 10000,
  'product': ['rosemary oil'],
  'type': ['body oil'],
  'brand': ['nature spell'],
  'characteristic': [],
  'color': [],
  'color_group': [],
  'age_group': [],
  'gender_group': [],
  'benefits': ['treats dry damaged hair', 'target hair loss'],
  'size': ['150ml'],
  'origin': ['uk']},
 {'index': 1252286,
  'title': 'Nizoral Anti-dandruff Shampoo, Treats and Prevents Dandruff, Suitable for Dry Flaky and Itchy Scalp, Contains Ketoconazole - 60ml',
  'stars': 4.6,
  'reviews': 27110,
  'price': 6.82,
  'boughtInLastMonth': 20000,
  'product': ['anti-dandruff shampoo'],
  'type': ['shampoo'],
  'brand': ['nizoral'],
  'characteristic': ['anti-dandruff',
   'suitable for dry flaky and itchy scalp'],
  'color': 

In [21]:
from neo4j import GraphDatabase

password = os.getenv("NEO4J_PASSWORD")

url = "bolt://localhost:7687"
username = "neo4j"
password = password


driver = GraphDatabase.driver(url, auth=(username, password))

In [22]:
entities_map = {
    "description": {
        "entity_name": "Description",
        "relationship_type": "HAS_DESCRIPTION"
    },
    "type": {
        "entity_name": "Type",
        "relationship_type": "HAS_TYPE"
    },
    "characteristic": {
        "entity_name": "Characteristic",
        "relationship_type": "HAS_CHARACTERISTIC"
    },
    "measurement": {
        "entity_name": "Measurement",
        "relationship_type": "HAS_MEASUREMENT"
    }, 
    "brand": {
        "entity_name": "Brand",
        "relationship_type": "HAS_BRAND"
        
    },
    "color": {
         "entity_name": "Color",
        "relationship_type": "HAS_COLOR"
    },
    "color_group": {
         "entity_name": "ColorGroup",
        "relationship_type": "HAS_COLOR_GROUP"
    },
    "age_group": {
        "entity_name": "AgeGroup",
        "relationship_type": "IS_FOR_AGE"

    },
    "gender_group": {
         "entity_name": "GenderGroup",
        "relationship_type": "IS_FOR_GENDER"
    },
    "benefits": {
         "entity_name": "Benefits",
        "relationship_type": "HAS_BENEFITS"
    },
    "size": {
         "entity_name": "Size",
        "relationship_type": "HAS_SIZE"
    },
    "origin": {
         "entity_name": "Origin",
        "relationship_type": "HAS_ORIGIN"
    },
}

In [23]:
def run_query(query, parameters=None):
    with driver.session() as session:
        result = session.run(query, parameters)
        return [r.data() for r in result]
    
def load_data(json_data):
    query = '''WITH $json_data as data
    MERGE (p:Product {
        index: data.index,
        title: data.title,
        stars: data.stars,
        reviews: data.reviews,
        price: data.price,
        boughtInLastMonth: data.boughtInLastMonth
    })
    '''
    for e in entities_map.keys():
        if e in json_data:
            query += f'''
            WITH p, data
            UNWIND {json_data[e]} as {e}
            MERGE ({e[:1]}{e[-1:]}:{entities_map[e]['entity_name']} {{value: {e}}})
            MERGE (p)-[:{entities_map[e]['relationship_type']}]->({e[:1]}{e[-1:]})
            '''
    run_query(query, {"json_data": json_data})

for data_entity in data_entities:
    load_data(data_entity)

In [24]:
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
from langchain_openai import OpenAIEmbeddings

vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(openai_api_key=api_key),
    url=url,
    username=username,
    password=password,
    index_name='person',
    node_label="Person",
    text_node_properties=['head', 'tail', 'relation'],
    embedding_node_property='embedding',
)

In [25]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

vector_qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(openai_api_key=api_key), chain_type="stuff", retriever=vector_index.as_retriever())

In [31]:
vector_qa.invoke(
    {"query": "What characteristics does apple cider gummies have?"}
)

{'query': 'What characteristics does apple cider gummies have?',
 'result': 'Apple cider gummies are a popular supplement that is made from apple cider vinegar. They are often infused with other ingredients like pectin, sugar, and natural flavors to make them more palatable. Apple cider gummies are known for potentially providing health benefits such as aiding digestion, boosting the immune system, and supporting weight loss.'}